## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-20-22-44-07 +0000,startribune,Live: Dave Matthews and Sierra Ferrell join L...,https://www.startribune.com/live-farm-aid-40-k...
1,2025-09-20-22-41-17 +0000,48hills,Rock legend Andy Summers takes aim with a diff...,https://48hills.org/2025/09/rock-legend-andy-s...
2,2025-09-20-22-38-30 +0000,nyt,Trump Justice Dept. Closed Investigation Into ...,https://www.nytimes.com/2025/09/20/us/politics...
3,2025-09-20-22-38-05 +0000,nyt,"Trump’s $100,000 H1-B Visa Fee Spurs Confusion...",https://www.nytimes.com/2025/09/20/business/tr...
4,2025-09-20-22-36-57 +0000,nypost,"Tech CEO falls 3,000 feet to his death while c...",https://nypost.com/2025/09/20/us-news/tech-ceo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,41
169,kirk,17
168,charlie,13
196,new,13
285,kimmel,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
195,2025-09-20-00-32-00 +0000,wsj,A top federal prosecutor who has been under pr...,https://www.wsj.com/politics/policy/trump-call...,115
187,2025-09-20-01-00-00 +0000,wsj,Charlie Kirk’s widow Erika Kirk will become th...,https://www.wsj.com/us-news/erika-kirk-charlie...,99
36,2025-09-20-19-50-05 +0000,nypost,White House rips Barack Obama for suggesting T...,https://nypost.com/2025/09/20/us-news/white-ho...,92
208,2025-09-19-23-29-42 +0000,nypost,Jimmy Kimmel foresaw his demise after second T...,https://nypost.com/2025/09/19/us-news/jimmy-ki...,90
204,2025-09-19-23-43-00 +0000,wsj,Trump announced a move to dramatically reshape...,https://www.wsj.com/politics/policy/trump-to-a...,89


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
195,115,2025-09-20-00-32-00 +0000,wsj,A top federal prosecutor who has been under pr...,https://www.wsj.com/politics/policy/trump-call...
187,86,2025-09-20-01-00-00 +0000,wsj,Charlie Kirk’s widow Erika Kirk will become th...,https://www.wsj.com/us-news/erika-kirk-charlie...
36,51,2025-09-20-19-50-05 +0000,nypost,White House rips Barack Obama for suggesting T...,https://nypost.com/2025/09/20/us-news/white-ho...
14,36,2025-09-20-21-33-56 +0000,nypost,High-speed police chase ends in deadly shootou...,https://nypost.com/2025/09/20/us-news/massive-...
9,35,2025-09-20-22-11-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
34,32,2025-09-20-20-04-18 +0000,nypost,White House tries easing panic over high-skill...,https://nypost.com/2025/09/20/us-news/white-ho...
91,32,2025-09-20-15-22-28 +0000,nypost,Q&A: NYC Mayor Eric Adams calls Jews his secre...,https://nypost.com/2025/09/20/us-news/q-amp-a-...
152,30,2025-09-20-08-21-49 +0000,nypost,"Russia launches large-scale attack on Ukraine,...",https://nypost.com/2025/09/20/world-news/russi...
208,27,2025-09-19-23-29-42 +0000,nypost,Jimmy Kimmel foresaw his demise after second T...,https://nypost.com/2025/09/19/us-news/jimmy-ki...
70,26,2025-09-20-17-16-35 +0000,nypost,"Last living US hostage, 21-year-old Edan Alexa...",https://nypost.com/2025/09/20/world-news/edan-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
